In [1]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
from tensorflow import keras

# prepare data

In [38]:
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()

unlabeld_index = np.ones(y_train.shape, np.bool)

N = 20
for i in range(10):
  idx = np.where(y_train == i)[0][:N]
  unlabeld_index[idx] = 0

x_unlabeld = x_train[np.where(unlabeld_index)[0], ...]

x_train = x_train[np.where(~unlabeld_index)[0], ...]
y_train = y_train[np.where(~unlabeld_index)[0], ...]

<ipython-input-38-2c2ed39229d7>:3: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  unlabeld_index = np.ones(y_train.shape, np.bool)


In [3]:
# examples of categorical crossentropy
cce = keras.losses.CategoricalCrossentropy()

# a labeled data from the second class
y_true = [[0, 1, 0, 0]]
y_pred = [[0.05, 0.95, 0, 0]]
print(cce(y_true, y_pred).numpy())

# an ulabeled data
y_true = [[0, 0, 0, 0]]
y_pred = [[0.05, 0.95, 0, 0]]
print(cce(y_true, y_pred).numpy())

# another ulabeled data
y_true = [[0, 0, 0, 0]]
y_pred = [[0.1, 0.4, 0.3, 0.2]]
print(cce(y_true, y_pred).numpy())

0.051293306
0.0
0.0


In [9]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten

## A

In [39]:

X_train = x_train.astype('float32')
X_test = x_test.astype('float32')

mean = np.mean(X_train, axis = (0,1,2,3))
std = np.std(X_train, axis = (0,1,2,3))

X_train = (X_train-mean)/(std + 1e-7)
X_test = (X_test-mean)/(std + 1e-7)

y_train_categorial = keras.utils.to_categorical(y_train,10)
y_test_categorial = keras.utils.to_categorical(y_test,10)


In [10]:
MN2 = MobileNetV2(include_top=False, input_shape=(32, 32, 3), weights=None)
flatten1 = Flatten()(MN2.layers[-1].output)
output1 = Dense(10, activation='softmax')(flatten1)
model1 = Model(inputs=MN2.inputs, outputs=output1)

model1.compile(optimizer=tf.keras.optimizers.Adam(),
               loss='categorical_crossentropy',
              metrics=['accuracy'])

In [11]:
model1.fit(x_train, y_train_categorial, batch_size=64, epochs=100)

Epoch 1/100
4/4 [==============================] - 13s 189ms/step - loss: 2.7318 - accuracy: 0.1150
Epoch 2/100
4/4 [==============================] - 0s 19ms/step - loss: 2.3887 - accuracy: 0.1750
Epoch 3/100
4/4 [==============================] - 0s 19ms/step - loss: 2.1092 - accuracy: 0.2900
Epoch 4/100
4/4 [==============================] - 0s 23ms/step - loss: 1.9770 - accuracy: 0.3550
Epoch 5/100
4/4 [==============================] - 0s 21ms/step - loss: 1.8674 - accuracy: 0.3600
Epoch 6/100
4/4 [==============================] - 0s 20ms/step - loss: 1.8509 - accuracy: 0.3750
Epoch 7/100
4/4 [==============================] - 0s 20ms/step - loss: 1.8194 - accuracy: 0.3950
Epoch 8/100
4/4 [==============================] - 0s 20ms/step - loss: 1.7072 - accuracy: 0.4650
Epoch 9/100
4/4 [==============================] - 0s 19ms/step - loss: 1.5034 - accuracy: 0.5150
Epoch 10/100
4/4 [==============================] - 0s 20ms/step - loss: 1.6615 - accuracy: 0.4650
Epoch 11/100
4/4 

In [12]:
print('Accuracy on Test Set :', model1.evaluate(x_test, y_test_categorial)[1] * 100)

313/313 [==============================] - 4s 8ms/step - loss: 2.3061 - accuracy: 0.1000
Accuracy on Test Set : 10.000000149011612


## B

In [13]:
from random import randint

rotation_degrees = [None, cv2.ROTATE_90_CLOCKWISE, cv2.ROTATE_90_COUNTERCLOCKWISE, cv2.ROTATE_180]
l = len(rotation_degrees)

x_rotate = np.zeros(x_unlabeld.shape)
y_rotate = np.zeros((x_unlabeld.shape[0],1))

for i in range(x_unlabeld.shape[0]):
  r = randint(0, l-1)
  if r>0:
    y_rotate[i,0] = r
    x_rotate[i] = cv2.rotate(x_unlabeld[i], rotation_degrees[r])
  else:
    y_rotate[i,0] = 0
    x_rotate[i] = x_unlabeld[i]

y_rotate_categorical = tf.keras.utils.to_categorical(y_rotate)

In [14]:
MN2 = MobileNetV2(include_top=False, input_shape=(32, 32, 3), weights=None)
flatten2 = Flatten()(MN2.layers[-1].output)
output2 = Dense(4, activation='softmax')(flatten2)

model2 = Model(inputs=MN2.inputs, outputs=output2)

model2.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
               loss='categorical_crossentropy',
               metrics=['accuracy'])

In [15]:
model2.fit(x_rotate, y_rotate_categorical, batch_size=64, epochs=50)

Epoch 1/50
779/779 [==============================] - 20s 21ms/step - loss: 1.5353 - accuracy: 0.2553
Epoch 2/50
779/779 [==============================] - 16s 20ms/step - loss: 1.3203 - accuracy: 0.3392
Epoch 3/50
779/779 [==============================] - 16s 21ms/step - loss: 1.1694 - accuracy: 0.4603
Epoch 4/50
779/779 [==============================] - 16s 21ms/step - loss: 1.0765 - accuracy: 0.5315
Epoch 5/50
779/779 [==============================] - 16s 21ms/step - loss: 1.0091 - accuracy: 0.5785
Epoch 6/50
779/779 [==============================] - 16s 20ms/step - loss: 0.9728 - accuracy: 0.5952
Epoch 7/50
779/779 [==============================] - 17s 22ms/step - loss: 0.9345 - accuracy: 0.6132
Epoch 8/50
779/779 [==============================] - 16s 20ms/step - loss: 0.9139 - accuracy: 0.6233
Epoch 9/50
779/779 [==============================] - 16s 20ms/step - loss: 0.8900 - accuracy: 0.6360
Epoch 10/50
779/779 [==============================] - 16s 20ms/step - loss: 0.868

In [24]:
output3 = Dense(10, activation='softmax')(model2.get_layer('flatten_3').output)
model3= Model(inputs=model2.input, outputs=output3)
model3.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
               loss='categorical_crossentropy',
               metrics=['accuracy'])

In [25]:
model3.fit(x_train, y_train_categorial, batch_size=64, epochs=100)

Epoch 1/100
4/4 [==============================] - 4s 26ms/step - loss: 2.3181 - accuracy: 0.0650
Epoch 2/100
4/4 [==============================] - 0s 28ms/step - loss: 2.3112 - accuracy: 0.0650
Epoch 3/100
4/4 [==============================] - 0s 22ms/step - loss: 2.3060 - accuracy: 0.0650
Epoch 4/100
4/4 [==============================] - 0s 21ms/step - loss: 2.2995 - accuracy: 0.0850
Epoch 5/100
4/4 [==============================] - 0s 20ms/step - loss: 2.2931 - accuracy: 0.0800
Epoch 6/100
4/4 [==============================] - 0s 21ms/step - loss: 2.2873 - accuracy: 0.0950
Epoch 7/100
4/4 [==============================] - 0s 18ms/step - loss: 2.2836 - accuracy: 0.1200
Epoch 8/100
4/4 [==============================] - 0s 18ms/step - loss: 2.2778 - accuracy: 0.1050
Epoch 9/100
4/4 [==============================] - 0s 19ms/step - loss: 2.2732 - accuracy: 0.1250
Epoch 10/100
4/4 [==============================] - 0s 20ms/step - loss: 2.2674 - accuracy: 0.1050
Epoch 11/100
4/4 [=

In [26]:
print('Accuracy on Test Set :', model3.evaluate(x_test, y_test)[1] * 100)

313/313 [==============================] - 3s 8ms/step - loss: 2.7250 - accuracy: 0.1606
Accuracy on Test Set : 16.06000065803528


## C

In [44]:
from sklearn.utils import shuffle

all_x_train = np.concatenate((x_train, x_unlabeld))
all_y_train = np.concatenate((y_train, np.zeros((x_unlabeld.shape[0], 1))))
all_x_train, all_y_train = shuffle(all_x_train, all_y_train)
all_y_train_categorical = tf.keras.utils.to_categorical(all_y_train, 10)

In [45]:
rotation_degrees = [None, cv2.ROTATE_90_CLOCKWISE, cv2.ROTATE_90_COUNTERCLOCKWISE, cv2.ROTATE_180]
l = len(rotation_degrees)

all_x_train_rotate = np.zeros(all_x_train.shape)
all_y_train_rotate = np.zeros((all_x_train.shape[0],1))

for i in range(all_x_train.shape[0]):
  r = randint(0, l-1)
  
  if r>0:
    all_y_train_rotate[i,0] = r
    all_x_train_rotate[i] = cv2.rotate(all_x_train[i], rotation_degrees[r])
  else:
    all_y_train_rotate[i,0] = 0
    all_x_train_rotate[i] = all_x_train[i]

all_y_train_rotate_categorical = tf.keras.utils.to_categorical(all_y_train_rotate, l)

In [46]:
x_test_rotate = np.zeros(x_test.shape)
y_test_rotate = np.zeros(y_test.shape)

for i in range(x_test.shape[0]):
  r = randint(0, l-1)
  
  if r>0:
    y_test_rotate[i,0] = r
    x_test_rotate[i] = cv2.rotate(x_test[i], rotation_degrees[r])
  else:
    y_test_rotate[i,0] = 0
    x_test_rotate[i] = x_test[i]

y_test_rotate_categorical = tf.keras.utils.to_categorical(y_test_rotate, l)

In [47]:
MN2 = MobileNetV2(include_top=False, input_shape=(32, 32, 3), weights=None)
flatten3 = Flatten()(MN2.layers[-1].output)
output3_1 = Dense(4, activation='softmax', name='rotation_output')(flatten3)
output3_2 = Dense(10, activation='softmax', name='classification_output')(flatten3)

model4 = Model(inputs=MN2.inputs, outputs=[output3_1, output3_2])

In [48]:
def train(class_coef, rotation_coef):
  model4.compile(
                optimizer=tf.keras.optimizers.Adam(learning_rate=0.01), 
                loss={
                    'classification_output': 'categorical_crossentropy', 
                    'rotation_output': 'categorical_crossentropy'},
                loss_weights={
                    'classification_output': class_coef, 
                    'rotation_output': rotation_coef},
                metrics={
                    'classification_output': 'accuracy',
                    'rotation_output': 'accuracy'}
  )

  model4.fit(
              all_x_train_rotate,
              [all_y_train_rotate_categorical, all_y_train_categorical], 
              validation_data=(x_test_rotate, [y_test_rotate_categorical, y_test_categorial]),
              batch_size=128, 
              epochs=20
  )

In [50]:
train(1,5)

Epoch 1/20
391/391 [==============================] - 16s 31ms/step - loss: 3.3191 - rotation_output_loss: 0.6572 - classification_output_loss: 0.0331 - rotation_output_accuracy: 0.7405 - classification_output_accuracy: 0.9964 - val_loss: 84.3674 - val_rotation_output_loss: 8.3291 - val_classification_output_loss: 42.7219 - val_rotation_output_accuracy: 0.4911 - val_classification_output_accuracy: 0.1000
Epoch 2/20
391/391 [==============================] - 10s 27ms/step - loss: 3.2100 - rotation_output_loss: 0.6356 - classification_output_loss: 0.0317 - rotation_output_accuracy: 0.7519 - classification_output_accuracy: 0.9964 - val_loss: 93.1861 - val_rotation_output_loss: 7.2456 - val_classification_output_loss: 56.9578 - val_rotation_output_accuracy: 0.5014 - val_classification_output_accuracy: 0.1000
Epoch 3/20
391/391 [==============================] - 11s 29ms/step - loss: 3.0836 - rotation_output_loss: 0.6105 - classification_output_loss: 0.0312 - rotation_output_accuracy: 0.761

In [51]:
train(1, 1)

Epoch 1/20
391/391 [==============================] - 17s 31ms/step - loss: 0.3623 - rotation_output_loss: 0.3293 - classification_output_loss: 0.0330 - rotation_output_accuracy: 0.8746 - classification_output_accuracy: 0.9964 - val_loss: 15.1454 - val_rotation_output_loss: 2.3821 - val_classification_output_loss: 12.7633 - val_rotation_output_accuracy: 0.6632 - val_classification_output_accuracy: 0.1000
Epoch 2/20
391/391 [==============================] - 10s 27ms/step - loss: 0.3456 - rotation_output_loss: 0.3144 - classification_output_loss: 0.0312 - rotation_output_accuracy: 0.8805 - classification_output_accuracy: 0.9964 - val_loss: 12.4999 - val_rotation_output_loss: 2.4007 - val_classification_output_loss: 10.0992 - val_rotation_output_accuracy: 0.6714 - val_classification_output_accuracy: 0.1000
Epoch 3/20
391/391 [==============================] - 11s 28ms/step - loss: 0.3323 - rotation_output_loss: 0.3016 - classification_output_loss: 0.0307 - rotation_output_accuracy: 0.885

In [52]:
train(4, 1)

Epoch 1/20
391/391 [==============================] - 16s 30ms/step - loss: 0.3305 - rotation_output_loss: 0.2018 - classification_output_loss: 0.0322 - rotation_output_accuracy: 0.9230 - classification_output_accuracy: 0.9964 - val_loss: 42.1855 - val_rotation_output_loss: 1.8730 - val_classification_output_loss: 10.0781 - val_rotation_output_accuracy: 0.6865 - val_classification_output_accuracy: 0.1000
Epoch 2/20
391/391 [==============================] - 11s 28ms/step - loss: 0.3177 - rotation_output_loss: 0.1967 - classification_output_loss: 0.0303 - rotation_output_accuracy: 0.9269 - classification_output_accuracy: 0.9964 - val_loss: 50.4147 - val_rotation_output_loss: 2.3530 - val_classification_output_loss: 12.0154 - val_rotation_output_accuracy: 0.6749 - val_classification_output_accuracy: 0.1000
Epoch 3/20
391/391 [==============================] - 11s 27ms/step - loss: 0.3000 - rotation_output_loss: 0.1830 - classification_output_loss: 0.0293 - rotation_output_accuracy: 0.930